In [1]:
import folium
import json
# import matplotlib.cm as cm
# import matplotlib.colors as colors
# import matlibplot.pyplot as plt
import numpy as np
import pandas as pd
# import seaborn as sns
import requests
from sklearn.cluster import KMeans

In [2]:
import pyproj

import math

def lonlat_to_xy(lon, lat):
    proj_latlon = pyproj.Proj(proj='latlong',datum='WGS84')
    proj_xy = pyproj.Proj(proj="utm", zone=48, datum='WGS84')
    xy = pyproj.transform(proj_latlon, proj_xy, lon, lat)
    return xy[0], xy[1]

def xy_to_lonlat(x, y):
    proj_latlon = pyproj.Proj(proj='latlong',datum='WGS84')
    proj_xy = pyproj.Proj(proj="utm", zone=48, datum='WGS84')
    lonlat = pyproj.transform(proj_xy, proj_latlon, x, y)
    return lonlat[0], lonlat[1]

def calc_xy_distance(x1, y1, x2, y2):
    dx = x2 - x1
    dy = y2 - y1
    return math.sqrt(dx*dx + dy*dy)

In [3]:
hcm_center = (10.762622, 106.660172)

In [4]:
hcm_center_x, hcm_center_y = lonlat_to_xy(hcm_center[1], hcm_center[0]) # City center in Cartesian coordinates

k = math.sqrt(3) / 2 # Vertical offset for hexagonal grid cells
x_min = hcm_center_x - 6000
x_step = 600
y_min = hcm_center_y - 6000 - (int(21/k)*k*600 - 12000)/2
y_step = 600 * k 

latitudes = []
longitudes = []
distances_from_center = []
xs = []
ys = []

for i in range(0, int(21/k)):
    y = y_min + i * y_step
    x_offset = 300 if i%2==0 else 0
    for j in range(0, 21):
        x = x_min + j * x_step + x_offset
        distance_from_center = calc_xy_distance(hcm_center_x, hcm_center_y, x, y)
        if (distance_from_center <= 6001):
            lon, lat = xy_to_lonlat(x, y)
            latitudes.append(lat)
            longitudes.append(lon)
            distances_from_center.append(distance_from_center)
            xs.append(x)
            ys.append(y)

In [5]:
map_hcm = folium.Map(location=hcm_center, zoom_start=13)

folium.Marker(hcm_center, popup='Alexanderplatz').add_to(map_hcm)

for lat, lon in zip(latitudes, longitudes):
    #folium.CircleMarker([lat, lon], radius=2, color='blue', fill=True, fill_color='blue', fill_opacity=1).add_to(map_hcm) 
    folium.Circle([lat, lon], radius=300, color='blue', fill=False).add_to(map_hcm)
    #folium.Marker([lat, lon]).add_to(map_hcm)

map_hcm

In [19]:
def get_urls(client_id, client_secret, version, latitidues, longitudes, category, radius=600, limit=100, time='any', day='any'):
    urls = [f'https://api.foursquare.com/v2/venues/explore?&client_id={client_id}&client_secret={client_secret}&v={version}&ll={lat},{long}&categoryId={category}&radius={radius}&limit={limit}&time={time}&day={day}' for lat, long in zip(latitudes, longitudes)]

    return urls


def request_api(urls):
    results = [requests.get(url).json() for url in urls]

    return results


def check_errors(api_results):
    errors = []
    for idx, result in enumerate(api_results):
        if result['meta']['code'] != 200:
            errors[version].append((idx, result))
    
    return errors


def remedy_errors(api_results, urls, errors, tries=5):
    while len(errors) > 0 and tries > 0:
        for error in errors:
            idx = error[0]
            url = urls[idx]
            
            new_result = requests.get(url).json()
            api_results[idx] = new_result
            
        errors = check_errors(api_results)
        tries -= 1
    
    return api_results


def get_venues(api_results):
    venues = []
    for result in api_results:
        if result['meta']['code'] == 200:
            for item in result['response']['groups'][0]['items']:
                venues.append(item['venue'])
    
    return venues


def get_distinct_venues(venues):
    venue_ids = []
    distinct_venues = []
    for venue in venues:
        venue_id = venue['id']
        if venue_id not in venue_ids:
            venue_ids.append(venue_id)
            distinct_venues.append(venue)
    
    return distinct_venues


def get_venue_coordinates(venues):
    coordinates = [(venue['location']['lat'], venue['location']['lng']) for venue in venues]

    return coordinates

In [6]:
import datetime as dt
from dateutil import rrule

start_date = dt.date(2013, 1, 1)
end_date = dt.date(2019, 1, 1)

drange = list(rrule.rrule(rrule.MONTHLY, dtstart=start_date, until=end_date))
drange = [f'{str(d.year)}{str(d.month).zfill(2)}{str(d.day).zfill(2)}' for d in drange]

In [7]:
client_id = 'VBT1KEJNYQMDORW3N55MCKB3S35RZJVEG3I42IN3SSCLMQZO' # your Foursquare ID
client_secret = 'ZW3MU1JZLNMKMIS4NBS1X4VPSMMGXRIW1LLXL5KC0AGEZMXY' # your Foursquare Secret
versions = '20200101' # Foursquare API version

In [9]:
food_category = '4d4b7105d754a06374d81259'

urls_food = get_urls(client_id, client_secret, versions, latitudes, longitudes, food_category)
results_food = request_api(urls_food)
errors_food = check_errors(results_food)

In [10]:
results_food = remedy_errors(results_food, urls_food, errors_food, tries=10)
print('Errors count:', len(check_errors(results_food)))

Errors count: 0


In [20]:
venues_food = get_distinct_venues(get_venues(results_food))

with open('results_food.json', 'w') as fp:
    json.dump(results_food, fp)

with open('venues_food.json', 'w') as fp:
    json.dump(venues_food, fp)

In [27]:
len(results_food)

364

In [47]:
cats = [venue['categories'][0]['id'] for venue in venues_food]
print(len(cats))
print(len(venues_food))

2157
2157


In [52]:
venues_food[0]

{'id': '4da3ab0b0f578cfad7a1be18',
 'name': 'Beefsteak Conic',
 'location': {'address': '3 Conic Dong Nam A Apartment, Nguyen Van Linh, Phong Phu, Binh Chanh',
  'lat': 10.709983676056243,
  'lng': 106.64184953159211,
  'labeledLatLngs': [{'label': 'display',
    'lat': 10.709983676056243,
    'lng': 106.64184953159211}],
  'distance': 209,
  'cc': 'VN',
  'state': 'Thành phố Hồ Chí Minh',
  'country': 'Việt Nam',
  'formattedAddress': ['3 Conic Dong Nam A Apartment, Nguyen Van Linh, Phong Phu, Binh Chanh',
   'Thành phố Hồ Chí Minh',
   'Việt Nam']},
 'categories': [{'id': '4bf58dd8d48988d143941735',
   'name': 'Breakfast Spot',
   'pluralName': 'Breakfast Spots',
   'shortName': 'Breakfast',
   'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/breakfast_',
    'suffix': '.png'},
   'primary': True}],
 'photos': {'count': 0, 'groups': []}}